In [6]:
import matplotlib.pylab as plt
# import networkx as nx
import numpy as np
import pandas as pd
import time
import sys
import os
import datetime

In [8]:
# preamble for jupyter notebook and django
import django

sys.path.append('/home/leey/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
from parliament.tasks import do_search, run_tm
import cities.models as cmodels
from django.contrib.auth.models import User
import tmv_app.models as tm
from utils.tm_mgmt import update_topic_scores
from django.db.models import Q, Count, Func, F, Sum, Avg, Value as V

from utils.topic_comparison import compare_topic_queryset, save_res, sort_matrix, draw_score_matrix
from utils.topic_comparison import save_topic_list_as_table, draw_bipartite_topic_graph, bipartite_graph_from_matrix

In [3]:
docs = pm.Document.objects.filter(parlperiod__n=17, text_source="from https://www.bundestag.de/service/opendata "
                                              +"(scans of pdfs with xml metadata)" )
print(docs.count())

253


In [4]:
#user1 = User.objects.create_user('finn', 'mueller-hansen@mcc-berlin.net', 'mcc123')
user1, created =  User.objects.get_or_create(username='leey', email='lee@mcc-berlin.net')
print(user1)
user1.save()

leey


In [5]:
# Note: saving of paragraph and utterance count not working properly, only on the second call of do_search!
# pm.Search.objects.all().delete()

In [7]:
# simple search for utterances
ut_search_tei, created = pm.Search.objects.get_or_create(
                title="Kohle tei utterance",
                text="kohle",
                creator=user1,
                document_source="GermaParlTEI",
                search_object_type=2)
ut_search_tei.save()
if created:
    do_search(ut_search_tei.id)
print(ut_search_tei.par_count)
print(ut_search_tei.utterance_count)
print(ut_search_tei.runstats_set.all())

35298
3284
<QuerySet [<RunStats: RunStats object (964)>]>


In [9]:
# static NMF 
# run_tm(ut_search_tei.id, 10)

In [ ]:
# dynamic NMF 
# run_tm(ut_search_tei.id, 50, method='DT')

In [11]:
# simple search for paragraphs
par_search_tei, created = pm.Search.objects.get_or_create(
                title="Kohle tei paragraph",
                text="kohle",
                creator=user1,
                document_source="GermaParlTEI",
                search_object_type=1)
par_search_tei.save()
if created:
    print("doing search")
do_search(par_search_tei.id)
print(par_search_tei.par_count)
print(par_search_tei.utterance_count)
print(par_search_tei.runstats_set.all())

4961 paragraphs with search kohle
4961
3284
4961
3284
<QuerySet []>


In [ ]:
# number of topics
K = [20,30,40,50]

In [13]:
# K= 20 to 50 for static LDA
for i in K:
    run_tm(par_search_tei.id, i, method='NM')

creating term frequency-inverse document frequency matrix (0.215179443359375)
save terms to db (18.933669090270996)
running matrix factorization with NMF (40.507906675338745)
Reconstruction error of nmf: 67.3514953070532
saving document topic matrix to db (61.06920552253723)
topic model run done (68.8557231426239)
creating term frequency-inverse document frequency matrix (0.18682217597961426)
save terms to db (18.957873821258545)
running matrix factorization with NMF (41.17347192764282)
Reconstruction error of nmf: 66.71248906843883
saving document topic matrix to db (67.48965978622437)
topic model run done (77.71578574180603)
creating term frequency-inverse document frequency matrix (0.20966100692749023)
save terms to db (18.981791496276855)
running matrix factorization with NMF (40.7155864238739)
Reconstruction error of nmf: 66.17439504086828
saving document topic matrix to db (72.71077275276184)
topic model run done (85.25193619728088)
creating term frequency-inverse document freque

In [14]:
# K= 20 to 50 for static LDA
for i in K:
    run_tm(par_search_tei.id, i, method='LD')

creating term frequency matrix (0.37505531311035156)
save terms to db (19.831410884857178)
running Latent Dirichlet Allocation (47.257975816726685)
saving document topic matrix to db (360.3576240539551)
topic model run done (375.2204215526581)
creating term frequency matrix (0.41091012954711914)
save terms to db (18.888075590133667)
running Latent Dirichlet Allocation (39.877817153930664)
saving document topic matrix to db (103.55408644676208)
topic model run done (127.19487071037292)
creating term frequency matrix (0.20089960098266602)
save terms to db (18.702800750732422)
running Latent Dirichlet Allocation (39.945536851882935)
saving document topic matrix to db (125.68475413322449)
topic model run done (156.14503693580627)
creating term frequency matrix (0.23378729820251465)
save terms to db (19.08131217956543)
running Latent Dirichlet Allocation (41.06908965110779)
saving document topic matrix to db (145.6504306793213)
topic model run done (185.45620894432068)


## Dynamic ##

## Running Comparison ##

In [10]:
% matplotlib inline

runs = [968, 969]
runs = RunStats.objects.filter(pk__in=runs).order_by('K')

res_tw, ss, score_matrix = compare_topic_queryset(runs)
#options = {'fname': "/home/leey/coal-discourse"}
#save_res([res_tw,ss], options)
res_tw

NameError: name 'RunStats' is not defined